In [1]:
import os
import platform

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# import matplotlib.pyplot as plt
import tqdm
import cv2
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.craft_detector import CRAFTDetector

In [2]:
folder_name = "train"
data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc\\{folder_name}'
output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_gt\\{folder_name}'
output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_filter\\{folder_name}'
output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_craft\\{folder_name}'

if platform.system() == "Linux":
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")


os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True)
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [3]:
detector = CRAFTDetector()
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

In [4]:
x_image_loader._info

,raw_filename,filename,category,box,label,polygon,x0,y0,x1,y1
0,221005_013853963_Camera_0.jpg,1Tainan_nan,1,"[(735, 1009), (735, 1042), (852, 1036), (852, ...",沙崙綠能科技示範場域,"POLYGON ((735 1009, 735 1042, 852 1036, 852 10...",735,1004,852,1042
1,221005_013853963_Camera_0.jpg,1Tainan_nan,1,"[(735, 1040), (735, 1073), (852, 1067), (852, ...",大台南會展中心,"POLYGON ((735 1040, 735 1073, 852 1067, 852 10...",735,1035,852,1073
2,221005_013853963_Camera_0.jpg,1Tainan_nan,1,"[(854, 974), (854, 1007), (971, 1001), (971, 9...",中央研究院南部院區,"POLYGON ((854 974, 854 1007, 971 1001, 971 969...",854,969,971,1007
3,221005_013853963_Camera_0.jpg,1Tainan_nan,1,"[(853, 1005), (853, 1038), (970, 1032), (970, ...",資安暨智慧科技研發大樓,"POLYGON ((853 1005, 853 1038, 970 1032, 970 10...",853,1000,970,1038
4,221005_013853963_Camera_0.jpg,1Tainan_nan,1,"[(854, 1035), (854, 1068), (971, 1062), (971, ...",臺灣智駕測試實驗室,"POLYGON ((854 1035, 854 1068, 971 1062, 971 10...",854,1030,971,1068
...,...,...,...,...,...,...,...,...,...,...
2222,220707_061123781_Camera_1.jpg,3Taoyuan_nan,3,"[(2052, 873), (2055, 924), (2281, 899), (2277,...",南崁竹圍,"POLYGON ((2052 873, 2055 924, 2281 899, 2277 8...",2052,842,2281,924
2223,220707_061123781_Camera_1.jpg,3Taoyuan_nan,1,"[(2055, 972), (2252, 956), (2246, 900), (2055,...",第一航廈,"POLYGON ((2055 972, 2252 956, 2246 900, 2055 9...",2055,900,2252,972
2224,220707_061123781_Camera_1.jpg,3Taoyuan_nan,1,"[(2059, 1020), (2237, 1012), (2234, 955), (205...",二航入境,"POLYGON ((2059 1020, 2237 1012, 2234 955, 2057...",2057,955,2237,1020
2225,220707_061123781_Camera_1.jpg,3Taoyuan_nan,6,"[(2060, 1074), (2236, 1074), (2233, 1009), (20...",二航號停車場,"POLYGON ((2060 1074, 2236 1074, 2233 1009, 205...",2059,1009,2236,1074


In [5]:
global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "IOU"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    gt_image = x_image_loader[image_name]
    detected_image = detector(gt_image.image)
    if detected_image is None:
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    detected_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))

    for row_idx, row in overlapping_info.iterrows():
        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue
        
        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
        label_df.loc[global_index, "IOU"] = row["IOU"]

        crapped_img = detected_instance.image
        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
        global_index+=1
        

label_df.to_csv(os.path.join(output_folder, "labels.csv"), encoding = 'big5')

 46%|████████████████████████████████████▉                                           | 445/964 [04:14<04:56,  1.75it/s]


GEOSException: TopologyException: Input geom 0 is invalid: Self-intersection at or near point 1741.9801324503312 746.90728476821187 at 1741.9801324503312 746.90728476821187